# Mission to Mars
## Part 1: Initial scraping using Jupyter Notebook, BeautifulSoup, Pandas, and Requests/Splinter.

In [117]:
# import libraries
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import requests

In [118]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [119]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [120]:
# Nasa news URL
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

### Nasa Mars News

In [121]:
# HTML Object
html = browser.html

In [122]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

In [123]:
# Retrieve page with the requests 
news_title = soup.find('div', class_='content_title').find('a').text
news_p = soup.find('div', class_='article_teaser_body').text


In [124]:
# Print scrapped data 
print(news_title)
print(news_p)

NASA Social Media and Websites Win Webby Awards 
NASA's social media presence, the InSight mission social media accounts, NASA.gov and SolarSystem.NASA.gov will be honored at the 2019 Webby Awards - "the Oscars of the Internet."


### JPL Mars Space Images - Featured Image

In [125]:
# Use splinter to visit URL Images
featured_image = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(featured_image)

In [126]:
# HTML Object 
html = browser.html

In [127]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

In [128]:
# Retrieve background-image url from style tag 
featured_image_url  = soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]

In [129]:
# Website Url 
main_url = 'https://www.jpl.nasa.gov'

In [130]:
# Concat web url with scrapped route
featured_image_url = main_url + featured_image_url

# Display full link to featured image
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA16029-1920x1200.jpg'

### Mars Weather

In [131]:
# Mars Weather Twitter w/ splinter
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

In [132]:
# HTML Object 
html_weather = browser.html

#Beautiful Soup
soup = BeautifulSoup(html_weather, 'html.parser')


In [133]:
# Elements that have tweets
tweets = soup.find_all('div', class_='js-tweet-text-container')

# Retrieve all elements that contain news title in the specified range
# Look for entries that display weather related words to exclude non weather related tweets 
for tweet in tweets: 
    weather_tweet = tweet.find('p').text
    if 'Sol' and 'pressure' in weather_tweet:
        print(weather_tweet)
        break
    else: 
        pass
    
    

InSight sol 150 (2019-04-29) low -98.5ºC (-145.3ºF) high -20.4ºC (-4.6ºF)
winds from the SW at 4.6 m/s (10.4 mph) gusting to 14.4 m/s (32.1 mph)
pressure at 7.40 hPapic.twitter.com/Sn17diwg2t


### Mars Facts

In [134]:
#Mars facts url 
facts = 'http://space-facts.com/mars/'

# Parse url
mars_facts = pd.read_html(facts)


In [135]:
# Mars facts DF
mars = mars_facts[0]

# Assign columns
mars.columns = ['Description','Value']

# Set the index 
mars.set_index('Description', inplace=True)

# Save html code
mars.to_html()


# Display mars_df
mars

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


### Mars Hemispheres


In [136]:
# Hemispheres URL
hemispheres = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres)

In [137]:
# HTML Object
html_hemispheres = browser.html

# Parse with Beautiful Soup
soup = BeautifulSoup(html_hemispheres, 'html.parser')


In [138]:
# Retreive all hemispheres info
items = soup.find_all('div', class_='item')

# Create hemisphere list
hemispheres_image_urls = []

# Store
hemispheres_main_url = 'https://astrogeology.usgs.gov'



In [139]:
# Loop through stored items
for i in items: 
    # Store title
    title = i.find('h3').text
    
    # Store link that leads to full image website
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    
    # Visit the link that contains the full image website 
    browser.visit(hemispheres_main_url + partial_img_url)
    
    # HTML Object of each hemisphere web
    partial_img_html = browser.html
    
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup( partial_img_html, 'html.parser')
    
    # Retrieve full image source 
    img_url = hemispheres_main_url + soup.find('img', class_='wide-image')['src']
    
    # Append the retreived information into a list of dictionaries 
    hemispheres_image_urls.append({"title" : title, "img_url" : img_url})
    

# Display
hemispheres_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]